In [51]:
import torch
from torchvision import datasets, transforms
import numpy as np
from opacus import PrivacyEngine
from tqdm import tqdm

In [52]:
model = torch.nn.Sequential(torch.nn.Conv2d(1, 16, 8, 2, padding=3),
                            torch.nn.ReLU(),
                            torch.nn.MaxPool2d(2, 1), 
                            torch.nn.Conv2d(16, 32, 4, 2), 
                            torch.nn.ReLU(), 
                            torch.nn.MaxPool2d(2, 1), 
                            torch.nn.Flatten(), 
                            torch.nn.Linear(32 * 4 * 4, 32), 
                            torch.nn.ReLU(), 
                            torch.nn.Linear(32, 10))

optimizer = torch.optim.SGD(model.parameters(), lr=0.05)


In [74]:
target_delta = 0.00001
privacy_engine = PrivacyEngine(model, 
							   batch_size=1000, 
                               sample_size=60000,  
                               alphas=range(2,1000), 
                               noise_multiplier=25, 
                               max_grad_norm=1.0,
                               delta=target_delta
                              )

#privacy_engine.attach(optimizer)

/home/shahaichao/anaconda3/envs/dpsgd/lib/python3.8/site-packages/opacus/privacy_engine.py:517: UserWarning: The sample rate will be defined from ``batch_size`` and ``sample_size``.The returned privacy budget will be incorrect.
  warnings.warn(
/home/shahaichao/anaconda3/envs/dpsgd/lib/python3.8/site-packages/opacus/privacy_engine.py:194: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_rng`` turned on.
  warnings.warn(


In [75]:
eps, alpha = privacy_engine.get_privacy_spent()

In [76]:
eps, alpha

(0.01384319695186801, 999.0)

In [1]:
import math
import sys

import numpy as np
from scipy import special
import six


In [2]:
def _log_add(logx, logy):
  """Add two numbers in the log space."""
  a, b = min(logx, logy), max(logx, logy)
  if a == -np.inf:  # adding 0
    return b
  # Use exp(a) + exp(b) = (exp(a - b) + 1) * exp(b)
  return math.log1p(math.exp(a - b)) + b  # log1p(x) = log(x + 1)


def _log_sub(logx, logy):
  """Subtract two numbers in the log space. Answer must be non-negative."""
  if logx < logy:
    raise ValueError("The result of subtraction must be non-negative.")
  if logy == -np.inf:  # subtracting 0
    return logx
  if logx == logy:
    return -np.inf  # 0 is represented as -np.inf in the log space.

  try:
    # Use exp(x) - exp(y) = (exp(x - y) - 1) * exp(y).
    return math.log(math.expm1(logx - logy)) + logy  # expm1(x) = exp(x) - 1
  except OverflowError:
    return logx


def _log_print(logx):
  """Pretty print."""
  if logx < math.log(sys.float_info.max):
    return "{}".format(math.exp(logx))
  else:
    return "exp({})".format(logx)


def _compute_log_a_int(q, sigma, alpha):
  """Compute log(A_alpha) for integer alpha. 0 < q < 1."""
  assert isinstance(alpha, six.integer_types)

  # Initialize with 0 in the log space.
  log_a = -np.inf

  for i in range(alpha + 1):
    log_coef_i = (
        math.log(special.binom(alpha, i)) + i * math.log(q) +
        (alpha - i) * math.log(1 - q))

    s = log_coef_i + (i * i - i) / (2 * (sigma**2))
    log_a = _log_add(log_a, s)

  return float(log_a)


def _compute_log_a_frac(q, sigma, alpha):
  """Compute log(A_alpha) for fractional alpha. 0 < q < 1."""
  # The two parts of A_alpha, integrals over (-inf,z0] and [z0, +inf), are
  # initialized to 0 in the log space:
  log_a0, log_a1 = -np.inf, -np.inf
  i = 0

  z0 = sigma**2 * math.log(1 / q - 1) + .5

  while True:  # do ... until loop
    coef = special.binom(alpha, i)
    log_coef = math.log(abs(coef))
    j = alpha - i

    log_t0 = log_coef + i * math.log(q) + j * math.log(1 - q)
    log_t1 = log_coef + j * math.log(q) + i * math.log(1 - q)

    log_e0 = math.log(.5) + _log_erfc((i - z0) / (math.sqrt(2) * sigma))
    log_e1 = math.log(.5) + _log_erfc((z0 - j) / (math.sqrt(2) * sigma))

    log_s0 = log_t0 + (i * i - i) / (2 * (sigma**2)) + log_e0
    log_s1 = log_t1 + (j * j - j) / (2 * (sigma**2)) + log_e1

    if coef > 0:
      log_a0 = _log_add(log_a0, log_s0)
      log_a1 = _log_add(log_a1, log_s1)
    else:
      log_a0 = _log_sub(log_a0, log_s0)
      log_a1 = _log_sub(log_a1, log_s1)

    i += 1
    if max(log_s0, log_s1) < -30:
      break

  return _log_add(log_a0, log_a1)


def _compute_log_a(q, sigma, alpha):
  """Compute log(A_alpha) for any positive finite alpha."""
  if float(alpha).is_integer():
    return _compute_log_a_int(q, sigma, int(alpha))
  else:
    return _compute_log_a_frac(q, sigma, alpha)


def _log_erfc(x):
  """Compute log(erfc(x)) with high accuracy for large x."""
  try:
    return math.log(2) + special.log_ndtr(-x * 2**.5)
  except NameError:
    # If log_ndtr is not available, approximate as follows:
    r = special.erfc(x)
    if r == 0.0:
      # Using the Laurent series at infinity for the tail of the erfc function:
      #     erfc(x) ~ exp(-x^2-.5/x^2+.625/x^4)/(x*pi^.5)
      # To verify in Mathematica:
      #     Series[Log[Erfc[x]] + Log[x] + Log[Pi]/2 + x^2, {x, Infinity, 6}]
      return (-math.log(math.pi) / 2 - math.log(x) - x**2 - .5 * x**-2 +
              .625 * x**-4 - 37. / 24. * x**-6 + 353. / 64. * x**-8)
    else:
      return math.log(r)


def _compute_delta(orders, rdp, eps):
  """Compute delta given a list of RDP values and target epsilon.
  Args:
    orders: An array (or a scalar) of orders.
    rdp: A list (or a scalar) of RDP guarantees.
    eps: The target epsilon.
  Returns:
    Pair of (delta, optimal_order).
  Raises:
    ValueError: If input is malformed.
  """
  orders_vec = np.atleast_1d(orders)
  rdp_vec = np.atleast_1d(rdp)

  if len(orders_vec) != len(rdp_vec):
    raise ValueError("Input lists must have the same length.")

  deltas = np.exp((rdp_vec - eps) * (orders_vec - 1))
  idx_opt = np.argmin(deltas)
  return min(deltas[idx_opt], 1.), orders_vec[idx_opt]


def _compute_eps(orders, rdp, delta):
  """Compute epsilon given a list of RDP values and target delta.
  Args:
    orders: An array (or a scalar) of orders.
    rdp: A list (or a scalar) of RDP guarantees.
    delta: The target delta.
  Returns:
    Pair of (eps, optimal_order).
  Raises:
    ValueError: If input is malformed.
  """
  orders_vec = np.atleast_1d(orders)
  rdp_vec = np.atleast_1d(rdp)

  if len(orders_vec) != len(rdp_vec):
    raise ValueError("Input lists must have the same length.")

  eps = rdp_vec - math.log(delta) / (orders_vec - 1)

  idx_opt = np.nanargmin(eps)  # Ignore NaNs
  return eps[idx_opt], orders_vec[idx_opt]


def _compute_rdp(q, sigma, alpha):
  """Compute RDP of the Sampled Gaussian mechanism at order alpha.
  Args:
    q: The sampling rate.
    sigma: The std of the additive Gaussian noise.
    alpha: The order at which RDP is computed.
  Returns:
    RDP at alpha, can be np.inf.
  """
  if q == 0:
    return 0

  if q == 1.:
    return alpha / (2 * sigma**2)

  if np.isinf(alpha):
    return np.inf

  return _compute_log_a(q, sigma, alpha) / (alpha - 1)


def compute_rdp(q, noise_multiplier, steps, orders):
  """Compute RDP of the Sampled Gaussian Mechanism.
  Args:
    q: The sampling rate.
    noise_multiplier: The ratio of the standard deviation of the Gaussian noise
        to the l2-sensitivity of the function to which it is added.
    steps: The number of steps.
    orders: An array (or a scalar) of RDP orders.
  Returns:
    The RDPs at all orders, can be np.inf.
  """
  if np.isscalar(orders):
    rdp = _compute_rdp(q, noise_multiplier, orders)
  else:
    rdp = np.array([_compute_rdp(q, noise_multiplier, order)
                    for order in orders])

  return rdp * steps


def get_privacy_spent(orders, rdp, target_eps=None, target_delta=None):
  """Compute delta (or eps) for given eps (or delta) from RDP values.
  Args:
    orders: An array (or a scalar) of RDP orders.
    rdp: An array of RDP values. Must be of the same length as the orders list.
    target_eps: If not None, the epsilon for which we compute the corresponding
              delta.
    target_delta: If not None, the delta for which we compute the corresponding
              epsilon. Exactly one of target_eps and target_delta must be None.
  Returns:
    eps, delta, opt_order.
  Raises:
    ValueError: If target_eps and target_delta are messed up.
  """
  if target_eps is None and target_delta is None:
    raise ValueError(
        "Exactly one out of eps and delta must be None. (Both are).")

  if target_eps is not None and target_delta is not None:
    raise ValueError(
        "Exactly one out of eps and delta must be None. (None is).")

  if target_eps is not None:
    delta, opt_order = _compute_delta(orders, rdp, target_eps)
    return target_eps, delta, opt_order
  else:
    eps, opt_order = _compute_eps(orders, rdp, target_delta)
    return eps, target_delta, opt_order


def compute_rdp_from_ledger(ledger, orders):
  """Compute RDP of Sampled Gaussian Mechanism from ledger.
  Args:
    ledger: A formatted privacy ledger.
    orders: An array (or a scalar) of RDP orders.
  Returns:
    RDP at all orders, can be np.inf.
  """
  total_rdp = np.zeros_like(orders, dtype=float)
  for sample in ledger:
    # Compute equivalent z from l2_clip_bounds and noise stddevs in sample.
    # See https://arxiv.org/pdf/1812.06210.pdf for derivation of this formula.
    effective_z = sum([
        (q.noise_stddev / q.l2_norm_bound)**-2 for q in sample.queries])**-0.5
    total_rdp += compute_rdp(
        sample.selection_probability, effective_z, 1, orders)
  return total_rdp

In [3]:
def loop_for_sigma(q, T, eps, delta, cur_sigma, interval, rdp_orders=128, rgp=True):
    
    while True:
        orders = np.arange(2, rdp_orders, 0.1)
        steps = T
        if(rgp):
            rdp = compute_rdp(q, cur_sigma, steps, orders) * 2 ## when using residual gradients, the sensitivity is sqrt(2)
        else:
            rdp = compute_rdp(q, cur_sigma, steps, orders)
            print("cur_sigma:", cur_sigma)
            print("interval:", interval)
        cur_eps, _, opt_order = get_privacy_spent(orders, rdp, target_delta=delta)
        print("cur_eps:", cur_eps)
        print("eps:", eps)
        if(cur_eps<eps and cur_sigma>interval):
            cur_sigma -= interval
            previous_eps = cur_eps
        else:
            cur_sigma += interval
            break
            #return cur_sigma, _    
    return cur_sigma, previous_eps


## interval: init search inerval
## rgp: use residual gradient perturbation or not
def get_sigma(q, T, eps, delta, init_sigma=100, interval=1., rgp=True):
    cur_sigma = init_sigma
    
    cur_sigma, _ = loop_for_sigma(q, T, eps, delta, cur_sigma, interval, rgp=rgp)
    interval /= 100
    cur_sigma, _ = loop_for_sigma(q, T, eps, delta, cur_sigma, interval, rgp=rgp)
    interval /= 100
    cur_sigma, previous_eps = loop_for_sigma(q, T, eps, delta, cur_sigma, interval, rgp=rgp)
    return cur_sigma, previous_eps

In [26]:
eps = 0.5
sampling_prob = 64/5000
steps = 80/sampling_prob
delta = 0.00001

In [27]:
sigma, eps = get_sigma(sampling_prob, steps, eps, delta, rgp=False)

cur_sigma: 100
interval: 1.0
cur_eps: 0.09727424247843615
eps: 0.5
cur_sigma: 99.0
interval: 1.0
cur_eps: 0.09740725933424682
eps: 0.5
cur_sigma: 98.0
interval: 1.0
cur_eps: 0.09754437005754403
eps: 0.5
cur_sigma: 97.0
interval: 1.0
cur_eps: 0.09768574439712609
eps: 0.5
cur_sigma: 96.0
interval: 1.0
cur_eps: 0.09783156099264581
eps: 0.5
cur_sigma: 95.0
interval: 1.0
cur_eps: 0.0979820079394018
eps: 0.5
cur_sigma: 94.0
interval: 1.0
cur_eps: 0.09813728339538391
eps: 0.5
cur_sigma: 93.0
interval: 1.0
cur_eps: 0.09829759623431453
eps: 0.5
cur_sigma: 92.0
interval: 1.0
cur_eps: 0.09846316674868524
eps: 0.5
cur_sigma: 91.0
interval: 1.0
cur_eps: 0.09863422740714904
eps: 0.5
cur_sigma: 90.0
interval: 1.0
cur_eps: 0.09881102367125824
eps: 0.5
cur_sigma: 89.0
interval: 1.0
cur_eps: 0.09899381487679641
eps: 0.5
cur_sigma: 88.0
interval: 1.0
cur_eps: 0.0991828751856542
eps: 0.5
cur_sigma: 87.0
interval: 1.0
cur_eps: 0.09937849461482871
eps: 0.5
cur_sigma: 86.0
interval: 1.0
cur_eps: 0.0995809801

cur_sigma: 9.878900000000005
interval: 0.0001
cur_eps: 0.4995695566114529
eps: 0.5
cur_sigma: 9.878800000000005
interval: 0.0001
cur_eps: 0.49957470411792615
eps: 0.5
cur_sigma: 9.878700000000006
interval: 0.0001
cur_eps: 0.4995798517830628
eps: 0.5
cur_sigma: 9.878600000000006
interval: 0.0001
cur_eps: 0.49958499960685576
eps: 0.5
cur_sigma: 9.878500000000006
interval: 0.0001
cur_eps: 0.49959014758932063
eps: 0.5
cur_sigma: 9.878400000000006
interval: 0.0001
cur_eps: 0.49959529573045736
eps: 0.5
cur_sigma: 9.878300000000007
interval: 0.0001
cur_eps: 0.4996004440302776
eps: 0.5
cur_sigma: 9.878200000000007
interval: 0.0001
cur_eps: 0.49960559248878567
eps: 0.5
cur_sigma: 9.878100000000007
interval: 0.0001
cur_eps: 0.49961074110598297
eps: 0.5
cur_sigma: 9.878000000000007
interval: 0.0001
cur_eps: 0.4996158898818868
eps: 0.5
cur_sigma: 9.877900000000007
interval: 0.0001
cur_eps: 0.49962103881649444
eps: 0.5
cur_sigma: 9.877800000000008
interval: 0.0001
cur_eps: 0.49962618790982016
eps: 

In [28]:
sigma

9.870600000000024

In [25]:
250/10000

0.025

In [ ]:
1.5-3.44
1.3-3.93
1 - 5
0.67-7
0.61-8.12
0.60-8.25
0.5 - 10
0.4 - 12
0.35 - 14
0.27 - 18

s:4 eps:1.8
s:7 eps:1
s:7.7 eps:0.9
s:8 eps:0.87
s:8.6 eps:0.8
s:

sigma:2 eps:4
sigma:4 eps:1.8
sigma:6 eps:1.18
sigma:8 eps:0.87
sigma:10 epoch:80 sample_rate:250/10000 eps:0.69 
sigma:14 eps:0.49  
sigma:18 eps:0.38
sigma:22 eps:0.31
sigma:26 eps:0.26
sigma:30 eps:0.23

### Dataset

In [5]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

import os

import numpy as np
from sklearn.random_projection import SparseRandomProjection, GaussianRandomProjection

from rdp_accountant import compute_rdp, get_privacy_spent

from lanczos import Lanczos

/home/shahaichao/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
def get_data_loader(dataset, batchsize):
    if(dataset == 'svhn'):
        transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
        ])
        trainset = torchvision.datasets.SVHN(root='./data/SVHN',split='train', download=False, transform=transform)
        trainloader = torch.utils.data.DataLoader(trainset, batch_size=73257, shuffle=True, num_workers=0) #load full btach into memory, to concatenate with extra data

        extraset = torchvision.datasets.SVHN(root='./data/SVHN',split='extra', download=False, transform=transform)
        extraloader = torch.utils.data.DataLoader(extraset, batch_size=531131, shuffle=True, num_workers=0) #load full btach into memory

        testset = torchvision.datasets.SVHN(root='./data/SVHN',split='test', download=False, transform=transform)
        testloader = torch.utils.data.DataLoader(testset, batch_size=batchsize, shuffle=False, num_workers=0)
        return trainloader, extraloader, testloader, len(trainset)+len(extraset), len(testset)

    if(dataset == 'mnist'):
        transform=transforms.Compose([
            transforms.ToTensor(),
            #transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
        ])
        trainset = torchvision.datasets.MNIST(root='./data',train=True, download=False, transform=transform)
        trainloader = torch.utils.data.DataLoader(trainset, batch_size=batchsize, shuffle=True, num_workers=2) #load full btach into memory, to concatenate with extra data

        testset = torchvision.datasets.MNIST(root='./data',train=False, download=False, transform=transform)
        testloader = torch.utils.data.DataLoader(testset, batch_size=batchsize, shuffle=False, num_workers=2)
        return trainloader, testloader, len(trainset), len(testset)

    elif(dataset == 'fmnist'):
        transform=transforms.Compose([
            transforms.ToTensor(),
            #transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
        ])
        trainset = torchvision.datasets.FashionMNIST(root='./data',train=True, download=False, transform=transform)
        trainloader = torch.utils.data.DataLoader(trainset, batch_size=batchsize, shuffle=True, num_workers=2) #load full btach into memory, to concatenate with extra data

        testset = torchvision.datasets.FashionMNIST(root='./data',train=False, download=False, transform=transform)
        testloader = torch.utils.data.DataLoader(testset, batch_size=batchsize, shuffle=False, num_workers=2)
        return trainloader, testloader, len(trainset), len(testset)

    else:
        transform_train = transforms.Compose([
        # transforms.RandomCrop(32, padding=4),
        # transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
        ])
        transform_test = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
        ])

        trainset = torchvision.datasets.CIFAR10(root='./data/CIFAR10', train=True, download=False, transform=transform_train) 
        trainloader = torch.utils.data.DataLoader(trainset, batch_size=batchsize, shuffle=True, num_workers=2)

        testset = torchvision.datasets.CIFAR10(root='./data/CIFAR10', train=False, download=False, transform=transform_test) 
        testloader = torch.utils.data.DataLoader(testset, batch_size=batchsize, shuffle=False, num_workers=2)
        return trainloader, testloader, len(trainset), len(testset)

In [13]:
trainloader, testloader, n_training, n_test = get_data_loader('fmnist', batchsize = 500)
train_samples, train_labels = None, None

In [14]:
for train_samples, train_labels in trainloader:
    break

In [15]:
train_samples.shape

torch.Size([500, 1, 28, 28])